Upload datasets from Stanford CS224N assignment2.

Note that google.colab.files.upload() would upload datasets onto your memories in Chrome!

In [0]:
#from google.colab import files

#uploaded = files.upload()

#for fn in uploaded.keys():
#    print('User uploaded file "{name}" with length {length} bytes'.format(
#        name=fn, length=len(uploaded[fn])))

Get free GPU~~~

In [44]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
#If so, you are ready to move on.
#Implement PyDrive to access data in google drive

!pip install -U -q PyDrive
 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
 
# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

reference tutorial: https://medium.com/deep-learning-turkey/google-colab-free-gpu-tutorial-e113627b9f5d

http://nali.org/load-google-drive-csv-panda-dataframe-google-colab/


In [0]:
##general_utils.py

import sys
import time
import numpy as np


def get_minibatches(data, minibatch_size, shuffle=True):
    """
    Iterates through the provided data one minibatch at at time. You can use this function to
    iterate through data in minibatches as follows:

        for inputs_minibatch in get_minibatches(inputs, minibatch_size):
            ...

    Or with multiple data sources:

        for inputs_minibatch, labels_minibatch in get_minibatches([inputs, labels], minibatch_size):
            ...

    Args:
        data: there are two possible values:
            - a list or numpy array
            - a list where each element is either a list or numpy array
        minibatch_size: the maximum number of items in a minibatch
        shuffle: whether to randomize the order of returned data
    Returns:
        minibatches: the return value depends on data:
            - If data is a list/array it yields the next minibatch of data.
            - If data a list of lists/arrays it returns the next minibatch of each element in the
              list. This can be used to iterate through multiple data sources
              (e.g., features and labels) at the same time.

    """
    list_data = type(data) is list and (type(data[0]) is list or type(data[0]) is np.ndarray)
    data_size = len(data[0]) if list_data else len(data)
    indices = np.arange(data_size)
    if shuffle:
        np.random.shuffle(indices)
    for minibatch_start in np.arange(0, data_size, minibatch_size):
        minibatch_indices = indices[minibatch_start:minibatch_start + minibatch_size]
        yield [_minibatch(d, minibatch_indices) for d in data] if list_data \
            else _minibatch(data, minibatch_indices)


def _minibatch(data, minibatch_idx):
    return data[minibatch_idx] if type(data) is np.ndarray else [data[i] for i in minibatch_idx]


def test_all_close(name, actual, expected):
    if actual.shape != expected.shape:
        raise ValueError("{:} failed, expected output to have shape {:} but has shape {:}"
                         .format(name, expected.shape, actual.shape))
    if np.amax(np.fabs(actual - expected)) > 1e-6:
        raise ValueError("{:} failed, expected {:} but value is {:}".format(name, expected, actual))
    else:
        print(name, "passed!")


In [47]:
##q1_softmax.py

import numpy as np
import tensorflow as tf

import os
#print(os.getcwd())
#os.chdir('/Users/KunWuYao/GitHub/NLP - Stanford/assignment2/')
#from utils.general_utils import test_all_close


def softmax(x):
    """
    Compute the softmax function in tensorflow.

    You might find the tensorflow functions tf.exp, tf.reduce_max,
    tf.reduce_sum, tf.expand_dims useful. (Many solutions are possible, so you may
    not need to use all of these functions). Recall also that many common
    tensorflow operations are sugared (e.g. x + y does elementwise addition
    if x and y are both tensors). Make sure to implement the numerical stability
    fixes as in the previous homework!

    Args:
        x:   tf.Tensor with shape (n_samples, n_features). Note feature vectors are
                  represented by row-vectors. (For simplicity, no need to handle 1-d
                  input as in the previous homework)
    Returns:
        out: tf.Tensor with shape (n_sample, n_features). You need to construct this
                  tensor in this problem.
    """

    ### YOUR CODE HERE
    a = tf.subtract(x, tf.reduce_max(x, 1, keep_dims = True))
    e = tf.exp(a)
    s = tf.reduce_sum(e,1, keep_dims = True)
    
    out = e / s
    ### END YOUR CODE

    return out


def cross_entropy_loss(y, yhat):
    """
    Compute the cross entropy loss in tensorflow.
    The loss should be summed over the current minibatch.

    y is a one-hot tensor of shape (n_samples, n_classes) and yhat is a tensor
    of shape (n_samples, n_classes). y should be of dtype tf.int32, and yhat should
    be of dtype tf.float32.

    The functions tf.to_float, tf.reduce_sum, and tf.log might prove useful. (Many
    solutions are possible, so you may not need to use all of these functions).

    Note: You are NOT allowed to use the tensorflow built-in cross-entropy
                functions.

    Args:
        y:    tf.Tensor with shape (n_samples, n_classes). One-hot encoded.
        yhat: tf.Tensorwith shape (n_sample, n_classes). Each row encodes a
                    probability distribution and should sum to 1.
    Returns:
        out:  tf.Tensor with shape (1,) (Scalar output). You need to construct this
                    tensor in the problem.
    """

    ### YOUR CODE HERE
    y = tf.to_float(y)
    out = -tf.reduce_sum(tf.multiply(y, tf.log(yhat)))
    
    ### END YOUR CODE

    return out


def test_softmax_basic():
    """
    Some simple tests of softmax to get you started.
    Warning: these are not exhaustive.
    """

    test1 = softmax(tf.constant(np.array([[1001, 1002], [3, 4]]), dtype=tf.float32))
    with tf.Session() as sess:
        test1 = sess.run(test1)
    test_all_close("Softmax test 1", test1, np.array([[0.26894142, 0.73105858],
                                                      [0.26894142, 0.73105858]]))

    test2 = softmax(tf.constant(np.array([[-1001, -1002]]), dtype=tf.float32))
    with tf.Session() as sess:
        test2 = sess.run(test2)
    test_all_close("Softmax test 2", test2, np.array([[0.73105858, 0.26894142]]))

    print("Basic (non-exhaustive) softmax tests pass\n")


def test_cross_entropy_loss_basic():
    """
    Some simple tests of cross_entropy_loss to get you started.
    Warning: these are not exhaustive.
    """
    y = np.array([[0, 1], [1, 0], [1, 0]])
    yhat = np.array([[.5, .5], [.5, .5], [.5, .5]])

    test1 = cross_entropy_loss(tf.constant(y, dtype=tf.int32), tf.constant(yhat, dtype=tf.float32))
    with tf.Session() as sess:
        test1 = sess.run(test1)
    expected = -3 * np.log(0.5)
    test_all_close("Cross-entropy test 1", test1, expected)

    print("Basic (non-exhaustive) cross-entropy tests pass")

if __name__ == "__main__":
    test_softmax_basic()
    test_cross_entropy_loss_basic()


Softmax test 1 passed!
Softmax test 2 passed!
Basic (non-exhaustive) softmax tests pass

Cross-entropy test 1 passed!
Basic (non-exhaustive) cross-entropy tests pass


In [0]:
##model.py
class Model(object):
    """Abstracts a Tensorflow graph for a learning task.

    We use various Model classes as usual abstractions to encapsulate tensorflow
    computational graphs. Each algorithm you will construct in this homework will
    inherit from a Model object.
    """
    def add_placeholders(self):
        """Adds placeholder variables to tensorflow computational graph.

        Tensorflow uses placeholder variables to represent locations in a
        computational graph where data is inserted.  These placeholders are used as
        inputs by the rest of the model building and will be fed data during
        training.

        See for more information:
        https://www.tensorflow.org/versions/r0.7/api_docs/python/io_ops.html#placeholders
        """
        raise NotImplementedError("Each Model must re-implement this method.")

    def create_feed_dict(self, inputs_batch, labels_batch=None):
        """Creates the feed_dict for one step of training.

        A feed_dict takes the form of:
        feed_dict = {
                <placeholder>: <tensor of values to be passed for placeholder>,
                ....
        }

        If labels_batch is None, then no labels are added to feed_dict.

        Hint: The keys for the feed_dict should be a subset of the placeholder
                    tensors created in add_placeholders.
        Args:
            inputs_batch: A batch of input data.
            labels_batch: A batch of label data.
        Returns:
            feed_dict: The feed dictionary mapping from placeholders to values.
        """
        raise NotImplementedError("Each Model must re-implement this method.")

    def add_prediction_op(self):
        """Implements the core of the model that transforms a batch of input data into predictions.

        Returns:
            pred: A tensor of shape (batch_size, n_classes)
        """
        raise NotImplementedError("Each Model must re-implement this method.")

    def add_loss_op(self, pred):
        """Adds Ops for the loss function to the computational graph.

        Args:
            pred: A tensor of shape (batch_size, n_classes)
        Returns:
            loss: A 0-d tensor (scalar) output
        """
        raise NotImplementedError("Each Model must re-implement this method.")

    def add_training_op(self, loss):
        """Sets up the training Ops.

        Creates an optimizer and applies the gradients to all trainable variables.
        The Op returned by this function is what must be passed to the
        sess.run() to train the model. See

        https://www.tensorflow.org/versions/r0.7/api_docs/python/train.html#Optimizer

        for more information.

        Args:
            loss: Loss tensor (a scalar).
        Returns:
            train_op: The Op for training.
        """

        raise NotImplementedError("Each Model must re-implement this method.")

    def train_on_batch(self, sess, inputs_batch, labels_batch):
        """Perform one step of gradient descent on the provided batch of data.

        Args:
            sess: tf.Session()
            input_batch: np.ndarray of shape (n_samples, n_features)
            labels_batch: np.ndarray of shape (n_samples, n_classes)
        Returns:
            loss: loss over the batch (a scalar)
        """
        feed = self.create_feed_dict(inputs_batch, labels_batch=labels_batch)
        _, loss = sess.run([self.train_op, self.loss], feed_dict=feed)
        return loss

    def predict_on_batch(self, sess, inputs_batch):
        """Make predictions for the provided batch of data

        Args:
            sess: tf.Session()
            input_batch: np.ndarray of shape (n_samples, n_features)
        Returns:
            predictions: np.ndarray of shape (n_samples, n_classes)
        """
        feed = self.create_feed_dict(inputs_batch)
        predictions = sess.run(self.pred, feed_dict=feed)
        return predictions

    def build(self):
        self.add_placeholders()
        self.pred = self.add_prediction_op()
        self.loss = self.add_loss_op(self.pred)
        self.train_op = self.add_training_op(self.loss)


In [74]:
#import time

#import numpy as np
#import tensorflow as tf

#from q1_softmax import softmax
#from q1_softmax import cross_entropy_loss
#from model import Model
#from utils.general_utils import get_minibatches


class Config1(object):
    """Holds model hyperparams and data information.

    The config class is used to store various hyperparameters and dataset
    information parameters. Model objects are passed a Config() object at
    instantiation. They can then call self.config.<hyperparameter_name> to
    get the hyperparameter settings.
    """
    n_samples = 1024
    n_features = 100
    n_classes = 5
    batch_size = 64
    n_epochs = 50
    lr = 1e-4
    


class SoftmaxModel(Model):
    """Implements a Softmax classifier with cross-entropy loss."""

    def add_placeholders(self):
        """Generates placeholder variables to represent the input tensors.

        These placeholders are used as inputs by the rest of the model building
        and will be fed data during training.

        Adds following nodes to the computational graph

        input_placeholder: Input placeholder tensor of shape
                                              (batch_size, n_features), type tf.float32
        labels_placeholder: Labels placeholder tensor of shape
                                              (batch_size, n_classes), type tf.int32

        Add these placeholders to self as the instance variables
            self.input_placeholder
            self.labels_placeholder
        """
        ### YOUR CODE HERE
        n_features = self.config1.n_features
        n_classes = self.config1.n_classes
        
        self.input_placeholder = tf.placeholder(tf.float32, shape = (None, n_features))
        self.labels_placeholder = tf.placeholder(tf.int32, shape = (None, n_classes))
        ### END YOUR CODE

    def create_feed_dict(self, inputs_batch, labels_batch=None):
        """Creates the feed_dict for training the given step.

        A feed_dict takes the form of:
        feed_dict = {
                <placeholder>: <tensor of values to be passed for placeholder>,
                ....
        }

        If label_batch is None, then no labels are added to feed_dict.

        Hint: The keys for the feed_dict should be the placeholder
                tensors created in add_placeholders.

        Args:
            inputs_batch: A batch of input data.
            labels_batch: A batch of label data.
        Returns:
            feed_dict: The feed dictionary mapping from placeholders to values.
        """
        ### YOUR CODE HERE
        feed_dict = {
            self.input_placeholder: inputs_batch,
            self.labels_placeholder: labels_batch
        }
        ### END YOUR CODE
        return feed_dict

    def add_prediction_op(self):
        """Adds the core transformation for this model which transforms a batch of input
        data into a batch of predictions. In this case, the transformation is a linear layer plus a
        softmax transformation:

        yhat = softmax(xW + b)

        Hint: The input x will be passed in through self.input_placeholder. Each ROW of
              self.input_placeholder is a single example. This is usually best-practice for
              tensorflow code.
        Hint: Make sure to create tf.Variables as needed.
        Hint: For this simple use-case, it's sufficient to initialize both weights W
                    and biases b with zeros.

        Returns:
            pred: A tensor of shape (batch_size, n_classes)
        """
        ### YOUR CODE HERE
        n_features = self.config1.n_features
        n_classes = self.config1.n_classes
        
        W = tf.Variable(tf.zeros([n_features, n_classes]))
        b = tf.Variable(tf.zeros([n_classes]))
        
        z = tf.add(tf.matmul(self.input_placeholder, W), b)
        pred = softmax(z)
        ### END YOUR CODE
        return pred

    def add_loss_op(self, pred):
        """Adds cross_entropy_loss ops to the computational graph.

        Hint: Use the cross_entropy_loss function we defined. This should be a very
                    short function.
        Args:
            pred: A tensor of shape (batch_size, n_classes)
        Returns:
            loss: A 0-d tensor (scalar)
        """
        ### YOUR CODE HERE
        loss = cross_entropy_loss(self.labels_placeholder, pred)
        ### END YOUR CODE
        return loss

    def add_training_op(self, loss):
        """Sets up the training Ops.

        Creates an optimizer and applies the gradients to all trainable variables.
        The Op returned by this function is what must be passed to the
        `sess.run()` call to cause the model to train. See

        https://www.tensorflow.org/api_docs/python/tf/train/Optimizer

        for more information. Use the learning rate from self.config.

        Hint: Use tf.train.GradientDescentOptimizer to get an optimizer object.
                    Calling optimizer.minimize() will return a train_op object.

        Args:
            loss: Loss tensor, from cross_entropy_loss.
        Returns:
            train_op: The Op for training.
        """
        ### YOUR CODE HERE
        optimizer = tf.train.GradientDescentOptimizer(self.config1.lr)
        train_op = optimizer.minimize(loss)
        ### END YOUR CODE
        return train_op

    def run_epoch(self, sess, inputs, labels):
        """Runs an epoch of training.

        Args:
            sess: tf.Session() object
            inputs: np.ndarray of shape (n_samples, n_features)
            labels: np.ndarray of shape (n_samples, n_classes)
        Returns:
            average_loss: scalar. Average minibatch loss of model on epoch.
        """
        n_minibatches, total_loss = 0, 0
        for input_batch, labels_batch in get_minibatches([inputs, labels], self.config1.batch_size):
            n_minibatches += 1
            total_loss += self.train_on_batch(sess, input_batch, labels_batch)
        return total_loss / n_minibatches

    def fit(self, sess, inputs, labels):
        """Fit model on provided data.

        Args:
            sess: tf.Session()
            inputs: np.ndarray of shape (n_samples, n_features)
            labels: np.ndarray of shape (n_samples, n_classes)
        Returns:
            losses: list of loss per epoch
        """
        losses = []
        for epoch in range(self.config1.n_epochs):
            start_time = time.time()
            average_loss = self.run_epoch(sess, inputs, labels)
            duration = time.time() - start_time
            print ('Epoch {:}: loss = {:.2f} ({:.3f} sec)'.format(epoch, average_loss, duration))
            losses.append(average_loss)
        return losses

    def __init__(self, config1):
        """Initializes the model.

        Args:
            config: A model configuration object of type Config
        """
        self.config1 = config1
        self.build()


def test_softmax_model():
    """Train softmax model for a number of steps."""
    config1 = Config1()

    # Generate random data to train the model on
    np.random.seed(1234)
    inputs = np.random.rand(config1.n_samples, config1.n_features)
    labels = np.zeros((config1.n_samples, config1.n_classes), dtype=np.int32)
    labels[:, 0] = 1

    # Tell TensorFlow that the model will be built into the default Graph.
    # (not required but good practice)
    with tf.Graph().as_default() as graph:
        # Build the model and add the variable initializer op
        model = SoftmaxModel(config1)
        init_op = tf.global_variables_initializer()
    # Finalizing the graph causes tensorflow to raise an exception if you try to modify the graph
    # further. This is good practice because it makes explicit the distinction between building and
    # running the graph.
    graph.finalize()

    # Create a session for running ops in the graph
    with tf.Session(graph=graph) as sess:
        # Run the op to initialize the variables.
        sess.run(init_op)
        # Fit the model
        losses = model.fit(sess, inputs, labels)

    # If ops are implemented correctly, the average loss should fall close to zero
    # rapidly.
    assert losses[-1] < .5
    print ("Basic (non-exhaustive) classifier tests pass")

if __name__ == "__main__":
    test_softmax_model()


Epoch 0: loss = 59.18 (0.052 sec)
Epoch 1: loss = 20.32 (0.032 sec)
Epoch 2: loss = 10.92 (0.032 sec)
Epoch 3: loss = 7.30 (0.036 sec)
Epoch 4: loss = 5.44 (0.032 sec)
Epoch 5: loss = 4.32 (0.035 sec)
Epoch 6: loss = 3.58 (0.039 sec)
Epoch 7: loss = 3.05 (0.032 sec)
Epoch 8: loss = 2.65 (0.030 sec)
Epoch 9: loss = 2.35 (0.035 sec)
Epoch 10: loss = 2.11 (0.032 sec)
Epoch 11: loss = 1.91 (0.029 sec)
Epoch 12: loss = 1.75 (0.030 sec)
Epoch 13: loss = 1.61 (0.035 sec)
Epoch 14: loss = 1.49 (0.030 sec)
Epoch 15: loss = 1.39 (0.029 sec)
Epoch 16: loss = 1.30 (0.037 sec)
Epoch 17: loss = 1.22 (0.032 sec)
Epoch 18: loss = 1.15 (0.030 sec)
Epoch 19: loss = 1.09 (0.028 sec)
Epoch 20: loss = 1.03 (0.036 sec)
Epoch 21: loss = 0.98 (0.031 sec)
Epoch 22: loss = 0.94 (0.031 sec)
Epoch 23: loss = 0.89 (0.028 sec)
Epoch 24: loss = 0.86 (0.030 sec)
Epoch 25: loss = 0.82 (0.029 sec)
Epoch 26: loss = 0.79 (0.033 sec)
Epoch 27: loss = 0.76 (0.038 sec)
Epoch 28: loss = 0.73 (0.031 sec)
Epoch 29: loss = 0.71

In [51]:
class PartialParse(object):
    def __init__(self, sentence):
        """Initializes this partial parse.

        Your code should initialize the following fields:
            self.stack: The current stack represented as a list with the top of the stack as the
                        last element of the list.
            self.buffer: The current buffer represented as a list with the first item on the
                         buffer as the first item of the list
            self.dependencies: The list of dependencies produced so far. Represented as a list of
                    tuples where each tuple is of the form (head, dependent).
                    Order for this list doesn't matter.

        The root token should be represented with the string "ROOT"

        Args:
            sentence: The sentence to be parsed as a list of words.
                      Your code should not modify the sentence.
        """
        # The sentence being parsed is kept for bookkeeping purposes. Do not use it in your code.
        self.sentence = sentence

        ### YOUR CODE HERE
        self.stack = ["ROOT"]
        self.buffer = [word for word in sentence]
        self.dependencies = []
        ### END YOUR CODE

    def parse_step(self, transition):
        """Performs a single parse step by applying the given transition to this partial parse

        Args:
            transition: A string that equals "S", "LA", or "RA" representing the shift, left-arc,
                        and right-arc transitions. You can assume the provided transition is a legal
                        transition.
        """
        ### YOUR CODE HERE
        if transition == 'S':
            self.stack.append(self.buffer.pop(0))
        elif transition == 'LA':
            dependent, head = self.stack[-2:]
            #Remove the dependent from the stack, keep the head.
            self.stack.pop(-2)
            self.dependencies.append((head, dependent))
        elif transition == 'RA':
            head, dependent = self.stack[-2:]
            self.stack.pop(-1)
            self.dependencies.append((head, dependent))
        ### END YOUR CODE

    def parse(self, transitions):
        """Applies the provided transitions to this PartialParse

        Args:
            transitions: The list of transitions in the order they should be applied
        Returns:
            dependencies: The list of dependencies produced when parsing the sentence. Represented
                          as a list of tuples where each tuple is of the form (head, dependent)
        """
        for transition in transitions:
            self.parse_step(transition)
        return self.dependencies


def minibatch_parse(sentences, model, batch_size):
    """Parses a list of sentences in minibatches using a model.

    Args:
        sentences: A list of sentences to be parsed (each sentence is a list of words)
        model: The model that makes parsing decisions. It is assumed to have a function
               model.predict(partial_parses) that takes in a list of PartialParses as input and
               returns a list of transitions predicted for each parse. That is, after calling
                   transitions = model.predict(partial_parses)
               transitions[i] will be the next transition to apply to partial_parses[i].
        batch_size: The number of PartialParses to include in each minibatch
    Returns:
        dependencies: A list where each element is the dependencies list for a parsed sentence.
                      Ordering should be the same as in sentences (i.e., dependencies[i] should
                      contain the parse for sentences[i]).
    """

    ### YOUR CODE HERE
    #[[sentence (stack, buffer, dependencies)], [sentence (stack, buffer, dependencies)]]
    partial_parses = [PartialParse(sentence) for sentence in sentences]
    unfinished_parses = partial_parses[:]
    
    while unfinished_parses:
        #Set minibatch size and content
        minibatch_parses = unfinished_parses[:batch_size]
        transitions = model.predict(minibatch_parses)
        
        #run through the minibatch items
        #if not done / no minibatch_parses deleted, keep running with the while loop!
        for parse, transition in zip(minibatch_parses, transitions):
            #Parse is a sentence (Class, has self.stack, self.buffer, self.dependencies)
            parse.parse_step(transition)
            if len(parse.stack) < 2 and len(parse.buffer) < 1:
                unfinished_parses.remove(parse)
    
    dependencies = [p.dependencies for p in partial_parses]
    ### END YOUR CODE

    return dependencies


def test_step(name, transition, stack, buf, deps,
              ex_stack, ex_buf, ex_deps):
    """Tests that a single parse step returns the expected output"""
    pp = PartialParse([])
    pp.stack, pp.buffer, pp.dependencies = stack, buf, deps

    pp.parse_step(transition)
    stack, buf, deps = (tuple(pp.stack), tuple(pp.buffer), tuple(sorted(pp.dependencies)))
    assert stack == ex_stack, \
        "{:} test resulted in stack {:}, expected {:}".format(name, stack, ex_stack)
    assert buf == ex_buf, \
        "{:} test resulted in buffer {:}, expected {:}".format(name, buf, ex_buf)
    assert deps == ex_deps, \
        "{:} test resulted in dependency list {:}, expected {:}".format(name, deps, ex_deps)
    print("{:} test passed!".format(name))


def test_parse_step():
    """Simple tests for the PartialParse.parse_step function
    Warning: these are not exhaustive
    """
    test_step("SHIFT", "S", ["ROOT", "the"], ["cat", "sat"], [],
              ("ROOT", "the", "cat"), ("sat",), ())
    test_step("LEFT-ARC", "LA", ["ROOT", "the", "cat"], ["sat"], [],
              ("ROOT", "cat",), ("sat",), (("cat", "the"),))
    test_step("RIGHT-ARC", "RA", ["ROOT", "run", "fast"], [], [],
              ("ROOT", "run",), (), (("run", "fast"),))


def test_parse():
    """Simple tests for the PartialParse.parse function
    Warning: these are not exhaustive
    """
    sentence = ["parse", "this", "sentence"]
    dependencies = PartialParse(sentence).parse(["S", "S", "S", "LA", "RA", "RA"])
    dependencies = tuple(sorted(dependencies))
    expected = (('ROOT', 'parse'), ('parse', 'sentence'), ('sentence', 'this'))
    assert dependencies == expected,  \
        "parse test resulted in dependencies {:}, expected {:}".format(dependencies, expected)
    assert tuple(sentence) == ("parse", "this", "sentence"), \
        "parse test failed: the input sentence should not be modified"
    print("parse test passed!")


class DummyModel(object):
    """Dummy model for testing the minibatch_parse function
    First shifts everything onto the stack and then does exclusively right arcs if the first word of
    the sentence is "right", "left" if otherwise.
    """
    def predict(self, partial_parses):
        return [("RA" if pp.stack[1] is "right" else "LA") if len(pp.buffer) == 0 else "S"
                for pp in partial_parses]


def test_dependencies(name, deps, ex_deps):
    """Tests the provided dependencies match the expected dependencies"""
    deps = tuple(sorted(deps))
    assert deps == ex_deps, \
        "{:} test resulted in dependency list {:}, expected {:}".format(name, deps, ex_deps)


def test_minibatch_parse():
    """Simple tests for the minibatch_parse function
    Warning: these are not exhaustive
    """
    sentences = [["right", "arcs", "only"],
                 ["right", "arcs", "only", "again"],
                 ["left", "arcs", "only"],
                 ["left", "arcs", "only", "again"]]
    deps = minibatch_parse(sentences, DummyModel(), 2)
    test_dependencies("minibatch_parse", deps[0],
                      (('ROOT', 'right'), ('arcs', 'only'), ('right', 'arcs')))
    test_dependencies("minibatch_parse", deps[1],
                      (('ROOT', 'right'), ('arcs', 'only'), ('only', 'again'), ('right', 'arcs')))
    test_dependencies("minibatch_parse", deps[2],
                      (('only', 'ROOT'), ('only', 'arcs'), ('only', 'left')))
    test_dependencies("minibatch_parse", deps[3],
                      (('again', 'ROOT'), ('again', 'arcs'), ('again', 'left'), ('again', 'only')))
    print("minibatch_parse test passed!")

if __name__ == '__main__':
    test_parse_step()
    test_parse()
    test_minibatch_parse()

SHIFT test passed!
LEFT-ARC test passed!
RIGHT-ARC test passed!
parse test passed!
minibatch_parse test passed!


In [52]:
#import numpy as np
#import tensorflow as tf


def xavier_weight_init():
    """Returns function that creates random tensor.

    The specified function will take in a shape (tuple or 1-d array) and
    returns a random tensor of the specified shape drawn from the
    Xavier initialization distribution.

    Hint: You might find tf.random_uniform useful.
    """
    def _xavier_initializer(shape, **kwargs):
        """Defines an initializer for the Xavier distribution.
        Specifically, the output should be sampled uniformly from [-epsilon, epsilon] where
            epsilon = sqrt(6) / <sum of the sizes of shape's dimensions>
        e.g., if shape = (2, 3), epsilon = sqrt(6 / (2 + 3))

        This function will be used as a variable initializer.

        Args:
            shape: Tuple or 1-d array that species the dimensions of the requested tensor.
        Returns:
            out: tf.Tensor of specified shape sampled from the Xavier distribution.
        """
        ### YOUR CODE HERE
        epsilon = tf.sqrt(6.0 / tf.cast(tf.reduce_sum(shape), tf.float32))
        out = tf.random_uniform(shape, -epsilon, epsilon)
        ### END YOUR CODE
        return out
    # Returns defined initializer function.
    return _xavier_initializer


def test_initialization_basic():
    """Some simple tests for the initialization.
    """
    print ("Running basic tests...")
    xavier_initializer = xavier_weight_init()
    shape = (1,)
    xavier_mat = xavier_initializer(shape)
    assert xavier_mat.get_shape() == shape

    shape = (1, 2, 3)
    xavier_mat = xavier_initializer(shape)
    assert xavier_mat.get_shape() == shape
    print ("Basic (non-exhaustive) Xavier initialization tests pass")


if __name__ == "__main__":
    test_initialization_basic()

Running basic tests...
Basic (non-exhaustive) Xavier initialization tests pass


How to get file id/location in the GoogleDrive object!

https://stackoverflow.com/questions/48376580/google-colab-how-to-read-data-from-my-google-drive

In [96]:
file_list = drive.ListFile({'q': "'1lG4pz5tkM8qvuHs5tEfB0Ew7dZKnddTJ' in parents", 'maxResults': 50}).GetList()
print('Received %s files from Files.list()' % len(file_list)) # <= 10
for file1 in file_list:
    print('title: %s, id: %s' % (file1['title'], file1['id']))

Received 9 files from Files.list()
title: dev.conll, id: 1qQdaVA7vltM7k_mAEA7BfosExu7jEqR2
title: weight, id: 1CrxPRgbU-lQamwg--9FuuJWrDd5plC0z
title: weight, id: 16zfXFh4xDR_GB1K_X_aG_wJMfsFK5v8T
title: train.gold.conll, id: 1DAh9a83MRgqv95eaj3LTUZA4KLK2zlY_
title: train.conll, id: 155yq13fVwHypPVicbZdtOigSu8sQt19z
title: test.gold.conll, id: 1eXvxO5y52kZuE5P8Ruz8RuFKKVjD0aHi
title: test.conll, id: 1Dgy3UZNyB_beX6JSh_5HrkjaTD_R_ewc
title: en-cw.txt, id: 1stVKwKmn6lO7LQz8jPqcwBaRwCEyTV4-
title: dev.gold.conll, id: 119gX_LgivBhTcfkbBXpnNIJ0OUauq4j7


Load Google Drive CSV into Pandas DataFrame for Google Colaboratory

http://nali.org/load-google-drive-csv-panda-dataframe-google-colab/

In [0]:
#embedding_file = drive.CreateFile({'id': '1stVKwKmn6lO7LQz8jPqcwBaRwCEyTV4-'})
#embedding_file.GetContentFile('en-cw.txt')

#for line in embedding_file:
#    print(line)

In [0]:
#for line in open('en-cw.txt').readlines():
#    print(line)

In [0]:
##parser_utils.py
"""Utilities for training the dependency parser.
You do not need to read/understand this code
"""

#import time
#import os
import logging
from collections import Counter
#from utils.general_utils import get_minibatches
#from q2_parser_transitions import minibatch_parse

#import numpy as np


P_PREFIX = '<p>:'
L_PREFIX = '<l>:'
UNK = '<UNK>'
NULL = '<NULL>'
ROOT = '<ROOT>'


class Config2(object):
    language = 'english'
    with_punct = True
    unlabeled = True
    lowercase = True
    use_pos = True
    use_dep = True
    use_dep = use_dep and (not unlabeled)
    data_path = './data'
    train_file = drive.CreateFile({'id': '155yq13fVwHypPVicbZdtOigSu8sQt19z'})
    train_file.GetContentFile('train.conll')
    dev_file = drive.CreateFile({'id': '1qQdaVA7vltM7k_mAEA7BfosExu7jEqR2'})
    dev_file.GetContentFile('dev.conll')
    test_file = drive.CreateFile({'id': '1Dgy3UZNyB_beX6JSh_5HrkjaTD_R_ewc'})
    test_file.GetContentFile('test.conll')
    embedding_file = drive.CreateFile({'id': '1stVKwKmn6lO7LQz8jPqcwBaRwCEyTV4-'})
    embedding_file.GetContentFile('en-cw.txt')


class Parser(object):
    """Contains everything needed for transition-based dependency parsing except for the model"""

    def __init__(self, dataset):
        root_labels = list([l for ex in dataset
                           for (h, l) in zip(ex['head'], ex['label']) if h == 0])
        counter = Counter(root_labels)
        if len(counter) > 1:
            logging.info('Warning: more than one root label')
            logging.info(counter)
        self.root_label = counter.most_common()[0][0]
        deprel = [self.root_label] + list(set([w for ex in dataset
                                               for w in ex['label']
                                               if w != self.root_label]))
        tok2id = {L_PREFIX + l: i for (i, l) in enumerate(deprel)}
        tok2id[L_PREFIX + NULL] = self.L_NULL = len(tok2id)

        config2 = Config2()
        self.unlabeled = config2.unlabeled
        self.with_punct = config2.with_punct
        self.use_pos = config2.use_pos
        self.use_dep = config2.use_dep
        self.language = config2.language

        if self.unlabeled:
            trans = ['L', 'R', 'S']
            self.n_deprel = 1
        else:
            trans = ['L-' + l for l in deprel] + ['R-' + l for l in deprel] + ['S']
            self.n_deprel = len(deprel)

        self.n_trans = len(trans)
        self.tran2id = {t: i for (i, t) in enumerate(trans)}
        self.id2tran = {i: t for (i, t) in enumerate(trans)}

        # logging.info('Build dictionary for part-of-speech tags.')
        tok2id.update(build_dict([P_PREFIX + w for ex in dataset for w in ex['pos']],
                                  offset=len(tok2id)))
        tok2id[P_PREFIX + UNK] = self.P_UNK = len(tok2id)
        tok2id[P_PREFIX + NULL] = self.P_NULL = len(tok2id)
        tok2id[P_PREFIX + ROOT] = self.P_ROOT = len(tok2id)

        # logging.info('Build dictionary for words.')
        tok2id.update(build_dict([w for ex in dataset for w in ex['word']],
                                  offset=len(tok2id)))
        tok2id[UNK] = self.UNK = len(tok2id)
        tok2id[NULL] = self.NULL = len(tok2id)
        tok2id[ROOT] = self.ROOT = len(tok2id)

        self.tok2id = tok2id
        self.id2tok = {v: k for (k, v) in tok2id.items()}

        self.n_features = 18 + (18 if config2.use_pos else 0) + (12 if config2.use_dep else 0)
        self.n_tokens = len(tok2id)

    def vectorize(self, examples):
        vec_examples = []
        for ex in examples:
            word = [self.ROOT] + [self.tok2id[w] if w in self.tok2id
                                  else self.UNK for w in ex['word']]
            pos = [self.P_ROOT] + [self.tok2id[P_PREFIX + w] if P_PREFIX + w in self.tok2id
                                   else self.P_UNK for w in ex['pos']]
            head = [-1] + ex['head']
            label = [-1] + [self.tok2id[L_PREFIX + w] if L_PREFIX + w in self.tok2id
                            else -1 for w in ex['label']]
            vec_examples.append({'word': word, 'pos': pos,
                                 'head': head, 'label': label})
        return vec_examples

    def extract_features(self, stack, buf, arcs, ex):
        if stack[0] == "ROOT":
            stack[0] = 0

        def get_lc(k):
            return sorted([arc[1] for arc in arcs if arc[0] == k and arc[1] < k])

        def get_rc(k):
            return sorted([arc[1] for arc in arcs if arc[0] == k and arc[1] > k],
                          reverse=True)

        p_features = []
        l_features = []
        features = [self.NULL] * (3 - len(stack)) + [ex['word'][x] for x in stack[-3:]]
        features += [ex['word'][x] for x in buf[:3]] + [self.NULL] * (3 - len(buf))
        if self.use_pos:
            p_features = [self.P_NULL] * (3 - len(stack)) + [ex['pos'][x] for x in stack[-3:]]
            p_features += [ex['pos'][x] for x in buf[:3]] + [self.P_NULL] * (3 - len(buf))

        for i in range(2):
            if i < len(stack):
                k = stack[-i-1]
                lc = get_lc(k)
                rc = get_rc(k)
                llc = get_lc(lc[0]) if len(lc) > 0 else []
                rrc = get_rc(rc[0]) if len(rc) > 0 else []

                features.append(ex['word'][lc[0]] if len(lc) > 0 else self.NULL)
                features.append(ex['word'][rc[0]] if len(rc) > 0 else self.NULL)
                features.append(ex['word'][lc[1]] if len(lc) > 1 else self.NULL)
                features.append(ex['word'][rc[1]] if len(rc) > 1 else self.NULL)
                features.append(ex['word'][llc[0]] if len(llc) > 0 else self.NULL)
                features.append(ex['word'][rrc[0]] if len(rrc) > 0 else self.NULL)

                if self.use_pos:
                    p_features.append(ex['pos'][lc[0]] if len(lc) > 0 else self.P_NULL)
                    p_features.append(ex['pos'][rc[0]] if len(rc) > 0 else self.P_NULL)
                    p_features.append(ex['pos'][lc[1]] if len(lc) > 1 else self.P_NULL)
                    p_features.append(ex['pos'][rc[1]] if len(rc) > 1 else self.P_NULL)
                    p_features.append(ex['pos'][llc[0]] if len(llc) > 0 else self.P_NULL)
                    p_features.append(ex['pos'][rrc[0]] if len(rrc) > 0 else self.P_NULL)

                if self.use_dep:
                    l_features.append(ex['label'][lc[0]] if len(lc) > 0 else self.L_NULL)
                    l_features.append(ex['label'][rc[0]] if len(rc) > 0 else self.L_NULL)
                    l_features.append(ex['label'][lc[1]] if len(lc) > 1 else self.L_NULL)
                    l_features.append(ex['label'][rc[1]] if len(rc) > 1 else self.L_NULL)
                    l_features.append(ex['label'][llc[0]] if len(llc) > 0 else self.L_NULL)
                    l_features.append(ex['label'][rrc[0]] if len(rrc) > 0 else self.L_NULL)
            else:
                features += [self.NULL] * 6
                if self.use_pos:
                    p_features += [self.P_NULL] * 6
                if self.use_dep:
                    l_features += [self.L_NULL] * 6

        features += p_features + l_features
        assert len(features) == self.n_features
        return features

    def get_oracle(self, stack, buf, ex):
        if len(stack) < 2:
            return self.n_trans - 1

        i0 = stack[-1]
        i1 = stack[-2]
        h0 = ex['head'][i0]
        h1 = ex['head'][i1]
        l0 = ex['label'][i0]
        l1 = ex['label'][i1]

        if self.unlabeled:
            if (i1 > 0) and (h1 == i0):
                return 0
            elif (i1 >= 0) and (h0 == i1) and \
                 (not any([x for x in buf if ex['head'][x] == i0])):
                return 1
            else:
                return None if len(buf) == 0 else 2
        else:
            if (i1 > 0) and (h1 == i0):
                return l1 if (l1 >= 0) and (l1 < self.n_deprel) else None
            elif (i1 >= 0) and (h0 == i1) and \
                 (not any([x for x in buf if ex['head'][x] == i0])):
                return l0 + self.n_deprel if (l0 >= 0) and (l0 < self.n_deprel) else None
            else:
                return None if len(buf) == 0 else self.n_trans - 1

    def create_instances(self, examples):
        all_instances = []
        succ = 0
        for id, ex in enumerate(examples):
            n_words = len(ex['word']) - 1

            # arcs = {(h, t, label)}
            stack = [0]
            buf = [i + 1 for i in range(n_words)]
            arcs = []
            instances = []
            for i in range(n_words * 2):
                gold_t = self.get_oracle(stack, buf, ex)
                if gold_t is None:
                    break
                legal_labels = self.legal_labels(stack, buf)
                assert legal_labels[gold_t] == 1
                instances.append((self.extract_features(stack, buf, arcs, ex),
                                  legal_labels, gold_t))
                if gold_t == self.n_trans - 1:
                    stack.append(buf[0])
                    buf = buf[1:]
                elif gold_t < self.n_deprel:
                    arcs.append((stack[-1], stack[-2], gold_t))
                    stack = stack[:-2] + [stack[-1]]
                else:
                    arcs.append((stack[-2], stack[-1], gold_t - self.n_deprel))
                    stack = stack[:-1]
            else:
                succ += 1
                all_instances += instances

        return all_instances

    def legal_labels(self, stack, buf):
        labels = ([1] if len(stack) > 2 else [0]) * self.n_deprel
        labels += ([1] if len(stack) >= 2 else [0]) * self.n_deprel
        labels += [1] if len(buf) > 0 else [0]
        return labels

    def parse(self, dataset, eval_batch_size=5000):
        sentences = []
        sentence_id_to_idx = {}
        for i, example in enumerate(dataset):
            n_words = len(example['word']) - 1
            sentence = [j + 1 for j in range(n_words)]
            sentences.append(sentence)
            sentence_id_to_idx[id(sentence)] = i

        model = ModelWrapper(self, dataset, sentence_id_to_idx)
        dependencies = minibatch_parse(sentences, model, eval_batch_size)

        UAS = all_tokens = 0.0
        for i, ex in enumerate(dataset):
            head = [-1] * len(ex['word'])
            for h, t, in dependencies[i]:
                head[t] = h
            for pred_h, gold_h, gold_l, pos in \
                    zip(head[1:], ex['head'][1:], ex['label'][1:], ex['pos'][1:]):
                    assert self.id2tok[pos].startswith(P_PREFIX)
                    pos_str = self.id2tok[pos][len(P_PREFIX):]
                    if (self.with_punct) or (not punct(self.language, pos_str)):
                        UAS += 1 if pred_h == gold_h else 0
                        all_tokens += 1
        UAS /= all_tokens
        return UAS, dependencies


class ModelWrapper(object):
    def __init__(self, parser, dataset, sentence_id_to_idx):
        self.parser = parser
        self.dataset = dataset
        self.sentence_id_to_idx = sentence_id_to_idx

    def predict(self, partial_parses):
        mb_x = [self.parser.extract_features(p.stack, p.buffer, p.dependencies,
                                             self.dataset[self.sentence_id_to_idx[id(p.sentence)]])
                for p in partial_parses]
        mb_x = np.array(mb_x).astype('int32')
        mb_l = [self.parser.legal_labels(p.stack, p.buffer) for p in partial_parses]
        pred = self.parser.model.predict_on_batch(self.parser.session, mb_x)
        pred = np.argmax(pred + 10000 * np.array(mb_l).astype('float32'), 1)
        pred = ["S" if p == 2 else ("LA" if p == 0 else "RA") for p in pred]
        return pred


def read_conll(in_file, lowercase=False, max_example=None):
    examples = []
    with open(in_file) as f:
        word, pos, head, label = [], [], [], []
        for line in f.readlines():
            sp = line.strip().split('\t')
            if len(sp) == 10:
                if '-' not in sp[0]:
                    word.append(sp[1].lower() if lowercase else sp[1])
                    pos.append(sp[4])
                    head.append(int(sp[6]))
                    label.append(sp[7])
            elif len(word) > 0:
                examples.append({'word': word, 'pos': pos, 'head': head, 'label': label})
                word, pos, head, label = [], [], [], []
                if (max_example is not None) and (len(examples) == max_example):
                    break
        if len(word) > 0:
            examples.append({'word': word, 'pos': pos, 'head': head, 'label': label})
    return examples


def build_dict(keys, n_max=None, offset=0):
    count = Counter()
    for key in keys:
        count[key] += 1
    ls = count.most_common() if n_max is None \
        else count.most_common(n_max)

    return {w[0]: index + offset for (index, w) in enumerate(ls)}


def punct(language, pos):
    if language == 'english':
        return pos in ["''", ",", ".", ":", "``", "-LRB-", "-RRB-"]
    elif language == 'chinese':
        return pos == 'PU'
    elif language == 'french':
        return pos == 'PUNC'
    elif language == 'german':
        return pos in ["$.", "$,", "$["]
    elif language == 'spanish':
        # http://nlp.stanford.edu/software/spanish-faq.shtml
        return pos in ["f0", "faa", "fat", "fc", "fd", "fe", "fg", "fh",
                       "fia", "fit", "fp", "fpa", "fpt", "fs", "ft",
                       "fx", "fz"]
    elif language == 'universal':
        return pos == 'PUNCT'
    else:
        raise ValueError('language: %s is not supported.' % language)


def minibatches(data, batch_size):
    x = np.array([d[0] for d in data])
    y = np.array([d[2] for d in data])
    one_hot = np.zeros((y.size, 3))
    one_hot[np.arange(y.size), y] = 1
    return get_minibatches([x, one_hot], batch_size)


def load_and_preprocess_data(reduced=True):
    config2 = Config2()

    print ("Loading data...",)
    start = time.time()
    train_set = read_conll('train.conll',
                           lowercase=config2.lowercase)
    dev_set = read_conll('dev.conll',
                         lowercase=config2.lowercase)
    test_set = read_conll('test.conll',
                          lowercase=config2.lowercase)
    if reduced:
        train_set = train_set[:1000]
        dev_set = dev_set[:500]
        test_set = test_set[:500]
    print( "took {:.2f} seconds".format(time.time() - start))

    print( "Building parser...",)
    start = time.time()
    parser = Parser(train_set)
    print( "took {:.2f} seconds".format(time.time() - start))

    print( "Loading pretrained embeddings...",)
    start = time.time()
    word_vectors = {}
    for line in open('en-cw.txt').readlines():
        sp = line.strip().split()
        word_vectors[sp[0]] = [float(x) for x in sp[1:]]
    embeddings_matrix = np.asarray(np.random.normal(0, 0.9, (parser.n_tokens, 50)), dtype='float32')

    for token in parser.tok2id:
        i = parser.tok2id[token]
        if token in word_vectors:
            embeddings_matrix[i] = word_vectors[token]
        elif token.lower() in word_vectors:
            embeddings_matrix[i] = word_vectors[token.lower()]
    print( "took {:.2f} seconds".format(time.time() - start))

    print( "Vectorizing data...",)
    start = time.time()
    train_set = parser.vectorize(train_set)
    dev_set = parser.vectorize(dev_set)
    test_set = parser.vectorize(test_set)
    print( "took {:.2f} seconds".format(time.time() - start))

    print( "Preprocessing training data...",)
    start = time.time()
    train_examples = parser.create_instances(train_set)
    print( "took {:.2f} seconds".format(time.time() - start))

    return parser, embeddings_matrix, train_examples, dev_set, test_set,

if __name__ == '__main__':
    pass



In [109]:
import pickle as cPickle
#import os
#import time
#import tensorflow as tf

#from model import Model
#from q2_initialization import xavier_weight_init
#from utils.parser_utils import minibatches, load_and_preprocess_data


class Config3(object):
    """Holds model hyperparams and data information.

    The config class is used to store various hyperparameters and dataset
    information parameters. Model objects are passed a Config() object at
    instantiation. They can then call self.config.<hyperparameter_name> to
    get the hyperparameter settings.
    """
    n_features = 36
    n_classes = 3
    dropout = 0.5  # (p_drop in the handout)
    embed_size = 50
    hidden_size = 200
    batch_size = 1024
    n_epochs = 15
    lr = 0.0005


class ParserModel(Model):
    """
    Implements a feedforward neural network with an embedding layer and single hidden layer.
    This network will predict which transition should be applied to a given partial parse
    configuration.
    """

    def add_placeholders(self):
        """Generates placeholder variables to represent the input tensors

        These placeholders are used as inputs by the rest of the model building and will be fed
        data during training.  Note that when "None" is in a placeholder's shape, it's flexible
        (so we can use different batch sizes without rebuilding the model).

        Adds following nodes to the computational graph

        input_placeholder: Input placeholder tensor of  shape (None, n_features), type tf.int32
        labels_placeholder: Labels placeholder tensor of shape (None, n_classes), type tf.float32
        dropout_placeholder: Dropout value placeholder (scalar), type tf.float32

        Add these placeholders to self as the instance variables
            self.input_placeholder
            self.labels_placeholder
            self.dropout_placeholder

        (Don't change the variable names)
        """
        ### YOUR CODE HERE
        n_features = self.config3.n_features
        n_classes = self.config3.n_classes
        
        self.input_placeholder = tf.placeholder(tf.int32, shape = (None, n_features))
        self.labels_placeholder = tf.placeholder(tf.float32, shape = (None, n_classes))
        self.dropout_placeholder = tf.placeholder(tf.float32)
        ### END YOUR CODE

    def create_feed_dict(self, inputs_batch, labels_batch=None, dropout=0):
        """Creates the feed_dict for the dependency parser.

        A feed_dict takes the form of:

        feed_dict = {
                <placeholder>: <tensor of values to be passed for placeholder>,
                ....
        }


        Hint: The keys for the feed_dict should be a subset of the placeholder
                    tensors created in add_placeholders.
        Hint: When an argument is None, don't add it to the feed_dict.

        Args:
            inputs_batch: A batch of input data.
            labels_batch: A batch of label data.
            dropout: The dropout rate.
        Returns:
            feed_dict: The feed dictionary mapping from placeholders to values.
        """
        ### YOUR CODE HERE
        feed_dict = {
            self.input_placeholder: inputs_batch,
            self.dropout_placeholder: dropout
        }
        
        if labels_batch is not None:
            feed_dict[self.labels_placeholder] = labels_batch
            
        ### END YOUR CODE
        return feed_dict

    def add_embedding(self):
        """Adds an embedding layer that maps from input tokens (integers) to vectors and then
        concatenates those vectors:
            - Creates a tf.Variable and initializes it with self.pretrained_embeddings.
            - Uses the input_placeholder to index into the embeddings tensor, resulting in a
              tensor of shape (None, n_features, embedding_size).
            - Concatenates the embeddings by reshaping the embeddings tensor to shape
              (None, n_features * embedding_size).

        Hint: You might find tf.nn.embedding_lookup useful.
        Hint: You can use tf.reshape to concatenate the vectors. See following link to understand
            what -1 in a shape means.
            https://www.tensorflow.org/api_docs/python/tf/reshape

        Returns:
            embeddings: tf.Tensor of shape (None, n_features*embed_size)
        """
        ### YOUR CODE HERE
        n_features = self.config3.n_features
        embedding_size = self.config3.embed_size
        
        vocabulary = tf.Variable(self.pretrained_embeddings)
        embeddings = tf.nn.embedding_lookup(vocabulary, self.input_placeholder)
        #Shape: (None, n_features, embedding_size)
        #Need to do reshape/concatenate
        embeddings = tf.reshape(embeddings, (-1, n_features * embedding_size))
        
        ### END YOUR CODE
        return embeddings

    def add_prediction_op(self):
        """Adds the 1-hidden-layer NN:
            h = Relu(xW + b1)
            h_drop = Dropout(h, dropout_rate)
            pred = h_dropU + b2

        Note that we are not applying a softmax to pred. The softmax will instead be done in
        the add_loss_op function, which improves efficiency because we can use
        tf.nn.softmax_cross_entropy_with_logits

        Use the initializer from q2_initialization.py to initialize W and U (you can initialize b1
        and b2 with zeros)

        Hint: Note that tf.nn.dropout takes the keep probability (1 - p_drop) as an argument.
              Therefore the keep probability should be set to the value of
              (1 - self.dropout_placeholder)

        Returns:
            pred: tf.Tensor of shape (batch_size, n_classes)
        """

        x = self.add_embedding()
        ### YOUR CODE HERE
        #initializer to initialize W and U
        xavier_init = xavier_weight_init()
        
        n_features = self.config3.n_features
        n_classes = self.config3.n_classes
        embedding_size = self.config3.embed_size
        hidden_size = self.config3.hidden_size
        
        #Hidden_layer
        W = tf.Variable(xavier_init((n_features * embedding_size, hidden_size)))
        b1 = tf.Variable(tf.zeros((1, hidden_size)))
        z = tf.add(tf.matmul(x, W), b1)
        h = tf.nn.relu(z)
        
        h_drop = tf.nn.dropout(h, keep_prob = 1 - self.dropout_placeholder)
        
        #Output_layer
        U = tf.Variable(xavier_init((hidden_size, n_classes)))
        b2 = tf.Variable(tf.zeros((1, n_classes)))
        
        pred = tf.add(tf.matmul(h_drop, U), b2)
        
        ### END YOUR CODE
        return pred

    def add_loss_op(self, pred):
        """Adds Ops for the loss function to the computational graph.
        In this case we are using cross entropy loss.
        The loss should be averaged over all examples in the current minibatch.

        Hint: You can use tf.nn.softmax_cross_entropy_with_logits to simplify your
                    implementation. You might find tf.reduce_mean useful.
        Args:
            pred: A tensor of shape (batch_size, n_classes) containing the output of the neural
                  network before the softmax layer.
        Returns:
            loss: A 0-d tensor (scalar)
        """
        ### YOUR CODE HERE
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = self.labels_placeholder, 
                                                                      logits = pred))
        
        ### END YOUR CODE
        return loss

    def add_training_op(self, loss):
        """Sets up the training Ops.

        Creates an optimizer and applies the gradients to all trainable variables.
        The Op returned by this function is what must be passed to the
        `sess.run()` call to cause the model to train. See

        https://www.tensorflow.org/api_docs/python/tf/train/Optimizer

        for more information.

        Use tf.train.AdamOptimizer for this model.
        Use the learning rate from self.config.
        Calling optimizer.minimize() will return a train_op object.

        Args:
            loss: Loss tensor, from cross_entropy_loss.
        Returns:
            train_op: The Op for training.
        """
        ### YOUR CODE HERE
        optimizer = tf.train.AdamOptimizer(self.config3.lr)
        train_op = optimizer.minimize(loss)
        
        ### END YOUR CODE
        return train_op

    def train_on_batch(self, sess, inputs_batch, labels_batch):
        feed = self.create_feed_dict(inputs_batch, labels_batch=labels_batch,
                                     dropout=self.config3.dropout)
        _, loss = sess.run([self.train_op, self.loss], feed_dict=feed)
        return loss

    def run_epoch(self, sess, parser, train_examples, dev_set):
        n_minibatches = 1 + len(train_examples) / self.config3.batch_size
        prog = tf.keras.utils.Progbar(target=n_minibatches)
        for i, (train_x, train_y) in enumerate(minibatches(train_examples, self.config3.batch_size)):
            loss = self.train_on_batch(sess, train_x, train_y)
            prog.update(i + 1, [("train loss", loss)], force=i + 1 == n_minibatches)

        print("  Evaluating on dev set",)
        dev_UAS, _ = parser.parse(dev_set)
        print("- dev UAS: {:.2f}".format(dev_UAS * 100.0))
        return dev_UAS

    def fit(self, sess, saver, parser, train_examples, dev_set):
        best_dev_UAS = 0
        for epoch in range(self.config3.n_epochs):
            print( "Epoch {:} out of {:}".format(epoch + 1, self.config3.n_epochs))
            dev_UAS = self.run_epoch(sess, parser, train_examples, dev_set)
            if dev_UAS > best_dev_UAS:
                best_dev_UAS = dev_UAS
                if saver:
                    print( "New best dev UAS! Saving model in ./data/weights/parser.weights")
                    saver.save(sess, './data/weights/parser.weights')
            print()

    def __init__(self, config3, pretrained_embeddings):
        self.pretrained_embeddings = pretrained_embeddings
        self.config3 = config3
        self.build()


def main(debug=False):
    print( 80 * "=")
    print( "INITIALIZING")
    print( 80 * "=")
    config3 = Config3()
    parser, embeddings, train_examples, dev_set, test_set = load_and_preprocess_data(debug)
    if not os.path.exists('./data/weights/'):
        os.makedirs('./data/weights/')

    with tf.Graph().as_default() as graph:
        print( "Building model...",)
        start = time.time()
        model = ParserModel(config3, embeddings)
        parser.model = model
        init_op = tf.global_variables_initializer()
        saver = None if debug else tf.train.Saver()
        print( "took {:.2f} seconds\n".format(time.time() - start))
    graph.finalize()

    with tf.Session(graph=graph) as session:
        parser.session = session
        session.run(init_op)

        print( 80 * "=")
        print( "TRAINING")
        print( 80 * "=")
        model.fit(session, saver, parser, train_examples, dev_set)

        if not debug:
            print( 80 * "=")
            print( "TESTING")
            print( 80 * "=")
            print( "Restoring the best model weights found on the dev set")
            saver.restore(session, './data/weights/parser.weights')
            print( "Final evaluation on test set",)
            UAS, dependencies = parser.parse(test_set)
            print( "- test UAS: {:.2f}".format(UAS * 100.0))
            print( "Writing predictions")
            with open('q2_test.predicted.pkl', 'wb') as f:
                cPickle.dump(dependencies, f, -1)
            folder = '1CrxPRgbU-lQamwg--9FuuJWrDd5plC0z'
            f = drive.CreateFile({"parents": [{'title': 'q2_test.predicted.pkl', 
                                               "kind": "drive#fileLink", 
                                               "id": folder}]})
            f.SetContentFile('q2_test.predicted.pkl')
            f.Upload()
            
            print( "Done!")


if __name__ == '__main__':
    main()

INITIALIZING
Loading data...
took 2.02 seconds
Building parser...
took 1.65 seconds
Loading pretrained embeddings...
took 2.94 seconds
Vectorizing data...
took 1.65 seconds
Preprocessing training data...
took 56.68 seconds
Building model...
took 0.25 seconds

TRAINING
Epoch 1 out of 15
 823/1848 [============>.................] 823/1848 [============>.................] - ETA: 15s - train loss: 0.2351

1843/1848 [============================>.]1843/1848 [============================>.] - ETA: 0s - train loss: 0.1820  Evaluating on dev set
- dev UAS: 84.37
New best dev UAS! Saving model in ./data/weights/parser.weights

Epoch 2 out of 15
1455/1848 [======================>.......]1455/1848 [======================>.......] - ETA: 4s - train loss: 0.1156

1844/1848 [============================>.]1844/1848 [============================>.] - ETA: 0s - train loss: 0.1143  Evaluating on dev set
- dev UAS: 86.52
New best dev UAS! Saving model in ./data/weights/parser.weights

Epoch 3 out of 15
1741/1848 [===========================>..]1741/1848 [===========================>..] - ETA: 1s - train loss: 0.1003

1846/1848 [============================>.]1846/1848 [============================>.] - ETA: 0s - train loss: 0.1001  Evaluating on dev set
- dev UAS: 86.99
New best dev UAS! Saving model in ./data/weights/parser.weights

Epoch 4 out of 15
1739/1848 [===========================>..]1739/1848 [===========================>..] - ETA: 1s - train loss: 0.0917

1842/1848 [============================>.]1842/1848 [============================>.] - ETA: 0s - train loss: 0.0916  Evaluating on dev set
- dev UAS: 87.44
New best dev UAS! Saving model in ./data/weights/parser.weights

Epoch 5 out of 15
1814/1848 [============================>.]1814/1848 [============================>.] - ETA: 0s - train loss: 0.0854

...

1843/1848 [============================>.]1843/1848 [============================>.] - ETA: 0s - train loss: 0.0693  Evaluating on dev set
- dev UAS: 88.63
New best dev UAS! Saving model in ./data/weights/parser.weights

Epoch 10 out of 15
1705/1848 [==========================>...]1705/1848 [==========================>...] - ETA: 1s - train loss: 0.0666

1842/1848 [============================>.]1842/1848 [============================>.] - ETA: 0s - train loss: 0.0666  Evaluating on dev set
- dev UAS: 88.82
New best dev UAS! Saving model in ./data/weights/parser.weights

Epoch 11 out of 15
1797/1848 [============================>.]1797/1848 [============================>.] - ETA: 0s - train loss: 0.0638

1845/1848 [============================>.]1845/1848 [============================>.] - ETA: 0s - train loss: 0.0639  Evaluating on dev set
- dev UAS: 88.55

Epoch 12 out of 15
1848/1848 [============================>.]1848/1848 [============================>.] - ETA: 0s - train loss: 0.0616  Evaluating on dev set
- dev UAS: 89.07
New best dev UAS! Saving model in ./data/weights/parser.weights

Epoch 13 out of 15
 168/1848 [=>............................] 168/1848 [=>............................] - ETA: 1:04 - train loss: 0.0567

1848/1848 [============================>.]1848/1848 [============================>.] - ETA: 0s - train loss: 0.0595  Evaluating on dev set
- dev UAS: 88.73

Epoch 14 out of 15
1581/1848 [========================>.....]1581/1848 [========================>.....] - ETA: 3s - train loss: 0.0568

1843/1848 [============================>.]1843/1848 [============================>.] - ETA: 0s - train loss: 0.0569  Evaluating on dev set
- dev UAS: 88.87

Epoch 15 out of 15
1842/1848 [============================>.]1842/1848 [============================>.] - ETA: 0s - train loss: 0.0554  Evaluating on dev set
- dev UAS: 88.83

TESTING
Restoring the best model weights found on the dev set
INFO:tensorflow:Restoring parameters from ./data/weights/parser.weights
Final evaluation on test set
- test UAS: 89.43
Writing predictions
Done!


How to create files on Google Drive with Python (don't know how to set root path......)

In [0]:
'''
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

# Create a local file to upload.
with open('/tmp/to_upload.txt', 'w') as f:
    f.write('my sample file')

print('/tmp/to_upload.txt contains:')
!cat /tmp/to_upload.txt

# Upload the file to Drive. See:
#
# https://developers.google.com/drive/v3/reference/files/create
# https://developers.google.com/drive/v3/web/manage-uploads
from googleapiclient.http import MediaFileUpload

file_metadata = {
  'name': 'Sample file',
  'mimeType': 'text/plain'
}
media = MediaFileUpload('/tmp/to_upload.txt', 
                        mimetype='text/plain',
                        resumable=True)
created = drive_service.files().create(body=file_metadata,
                                       media_body=media,
                                       fields='id').execute()
print('File ID: {}'.format(created.get('id')))
'''

In [97]:
#for file1 in file_list:       
#    print ('title: %s, id: %s' % (file1['title'], file1['id']))

title: dev.conll, id: 1qQdaVA7vltM7k_mAEA7BfosExu7jEqR2
title: weight, id: 1CrxPRgbU-lQamwg--9FuuJWrDd5plC0z
title: weight, id: 16zfXFh4xDR_GB1K_X_aG_wJMfsFK5v8T
title: train.gold.conll, id: 1DAh9a83MRgqv95eaj3LTUZA4KLK2zlY_
title: train.conll, id: 155yq13fVwHypPVicbZdtOigSu8sQt19z
title: test.gold.conll, id: 1eXvxO5y52kZuE5P8Ruz8RuFKKVjD0aHi
title: test.conll, id: 1Dgy3UZNyB_beX6JSh_5HrkjaTD_R_ewc
title: en-cw.txt, id: 1stVKwKmn6lO7LQz8jPqcwBaRwCEyTV4-
title: dev.gold.conll, id: 119gX_LgivBhTcfkbBXpnNIJ0OUauq4j7


In [0]:
#https://raspberrypi.stackexchange.com/questions/14186/use-pydrive-to-upload-files-to-google-drive-folder
fid = '1CrxPRgbU-lQamwg--9FuuJWrDd5plC0z'
f = drive.CreateFile({"parents": [{"kind": "drive#fileLink", "id": fid}]})
f.SetContentFile( 'en-cw.txt' )
f.Upload()